# graph

> Fill in a module description here

In [ ]:
#| default_exp graph

In [ ]:
#| hide
from nbdev.showdoc import *



In [ ]:
#| export
import os
import tempfile
import networkx as nx
import tree_sitter_python as tspython

from glob import glob
from uuid import uuid4
from difflib import get_close_matches
from repo2graph.utils import clone_repo
from tree_sitter import Language, Parser

from repo2graph.utils import trace_exception

def add_node(g:nx.DiGraph, name:str, **kwargs):
    node_id = str(uuid4())
    g.add_node(node_id, name=name, **kwargs)
    return node_id

def get_node_name(g: nx.DiGraph, id:str):
    return g.nodes[id]['name']

def remove_duplicates(g: nx.DiGraph):
    # Create a dictionary to track nodes by their attributes
    seen = set()
    nodes_to_remove = []

    for node_id in g.nodes:
        node_attr = str(g.nodes[node_id])  # Convert node attributes to string for comparison
        if node_attr in seen:
            nodes_to_remove.append(node_id)
        else:
            seen.add(node_attr)

    # Remove the duplicate nodes
    g.remove_nodes_from(nodes_to_remove)

    return g

def add_nodes_by_source(g:nx.DiGraph, source_path:str, source:str):
    components = source_path.split('/')
    components = components[:-1] + [components[-1].replace('.py','')]

    # Traverse the components and add nodes
    current_node_id = None  # The root node is not predefined

    for i in range(len(components)):
        next_node = '/'.join(components[:i+1])
        next_node_id = add_node(g, next_node)
        if current_node_id:
            g.add_edge(current_node_id, next_node_id)
        current_node_id = next_node_id
        
    g.nodes[current_node_id]['source'] = source
    g = parse_py(g, current_node_id)
    g = remove_duplicates(g)
    return g

            

def build_nodes_by_path(path: str, base_dir: str, repo_name:str, g: nx.DiGraph = None):
    """Adds nodes to the graph based on the file path and adds source code for .py files."""
    if g is None:
        g = nx.DiGraph()

    # Remove the base_dir from the path using removeprefix (Python 3.9+)
    path = path.removeprefix(base_dir + '/')

    # Split the remaining path into components
    components = [repo_name] + path.split('/')
    components = components[:-1] + [components[-1].replace('.py','')]

    # Traverse the components and add nodes
    current_node = None  # The root node is not predefined
    current_node_id = None

    for i in range(len(components)):
        next_node = '/'.join(components[:i+1])
        node_type = 'directory' if i<(len(components)-1) else 'file'
        next_node_id = add_node(g, next_node, type=node_type)
        if current_node_id:
            g.add_edge(current_node_id, next_node_id)
        current_node_id = next_node_id

    try:
        with open(os.path.join(base_dir, path), 'r') as file:
            source_code = file.read()
        # Add the source code as an attribute to the final node
        g.nodes[current_node_id]['source'] = source_code
        g = parse_py(g, current_node_id)
    except Exception as e:
        print(f"Error reading file {path}")
        trace_exception(e)
    g = remove_duplicates(g)
    return g
        
def build_repo_graph(repo_url: str = None, repo_dir: str = None, g: nx.DiGraph = None, ignore_files=['setup.py']):
    """Clones the repository (if repo_url is provided) or uses the local directory (if repo_dir is provided),
    and adds nodes based on the files in the repo."""

    # Ensure that either repo_url or repo_dir is provided, but not both
    if bool(repo_url) == bool(repo_dir):
        raise ValueError("You must provide exactly one of 'repo_url' or 'repo_dir', not both.")

    # If no graph is provided, initialize a new one
    if g is None:
        g = nx.DiGraph()

    # Determine the repo name based on the URL or directory
    if repo_url:
        repo_name = repo_url.split('/')[-1].replace('.git', '')
    else:
        repo_name = os.path.basename(os.path.normpath(repo_dir))

    # Use the local directory if provided, otherwise clone the repo
    if repo_url:
        with tempfile.TemporaryDirectory() as tempdir:
            # Clone the repository into the temporary directory
            clone_repo(repo_url, tempdir)

            # Get all file paths in the repository (recursively)
            repo_file_paths = glob(f"{tempdir}/**/*.py", recursive=True)
            # Add nodes to the graph for each file path
            for file_path in repo_file_paths:
                if file_path.split('/')[-1] in ignore_files:
                    continue
                # Add the node based on the file path, using base_dir as the root
                g = build_nodes_by_path(file_path, repo_dir or tempdir, repo_name, g)
    else:
        repo_file_paths = glob(f"{repo_dir}/**/*.py", recursive=True)

        # Add nodes to the graph for each file path
        for file_path in repo_file_paths:
            if file_path.split('/')[-1] in ignore_files:
                continue
            # Add the node based on the file path, using base_dir as the root
            g = build_nodes_by_path(file_path, repo_dir or tempdir, repo_name, g)
    g = remove_duplicates(g)
    return g

def parse_py(g: nx.DiGraph, root_node_id: str):
    """
    Extract class, function, and import relationships from a syntax tree into a given graph.

    Args:
        g (nx.DiGraph): A pre-existing directed graph.
        root_node_id (str): root node id
    """
    PY_LANGUAGE = Language(tspython.language())
    parser = Parser(PY_LANGUAGE)
    tree = parser.parse(bytes(g.nodes[root_node_id]['source'], "utf8"))
    
    def get_source_text(node):
        """Extract the source code corresponding to a node."""
        return bytes(g.nodes[root_node_id]['source'], "utf8")[node.start_byte:node.end_byte].decode("utf8")
    
    def traverse(treesitter_node, current_parent_id):
        if treesitter_node.type in ['class_definition', 'function_definition']:
            name_node = treesitter_node.child_by_field_name('name')
            if name_node:
                name = bytes(g.nodes[root_node_id]['source'], "utf8")[name_node.start_byte:name_node.end_byte].decode("utf8")
                full_name = '::'.join([get_node_name(g, current_parent_id), name])  # Construct hierarchical name
                source_text = get_source_text(treesitter_node)
                node_type = 'class' if treesitter_node.type == 'class_definition' else 'function'
                source_lines = g.nodes[root_node_id]['source'].splitlines()
                current_line = source_lines[treesitter_node.start_point[0]]
                indent = len(current_line) - len(current_line.lstrip(' '))
                
                # Get start and end lines for function definitions
                start_line = treesitter_node.start_point[0] + 1  # Adding 1 because line numbers are 1-based
                end_line = treesitter_node.end_point[0] + 1
                
                # Add node with attributes
                node_attributes = {
                    'type': node_type,
                    'parent_name': get_node_name(g, current_parent_id),
                    'parent_id': current_parent_id,
                    'source': source_text,
                    'indent': indent
                }
                
                if node_type == 'function':  # Only functions get start_line and end_line
                    node_attributes['start_line'] = start_line
                    node_attributes['end_line'] = end_line
                
                node_id = add_node(g, full_name, **node_attributes)
                
                # Connect to the parent if present
                if current_parent_id:
                    g.add_edge(current_parent_id, node_id)

                # If it's a class, traverse its body to extract methods
                if node_type == 'class':
                    body_node = treesitter_node.child_by_field_name('body')
                    if body_node:  # Recursively traverse the class body
                        traverse(body_node, node_id)

            # Set current_parent to full_name for child functions or classes
            current_parent_id = node_id if treesitter_node.type == 'class_definition' else current_parent_id
        
        # Recursively traverse other children
        for child in treesitter_node.children:
            traverse(child, current_parent_id)
    
    traverse(tree.root_node, root_node_id)
    g = remove_duplicates(g)
    return g

def find_nodes_by_path(g: nx.DiGraph, path: str, node_type=None):
    path = path.replace('.py', '')
    node_ids = []
    for node_id in g.nodes:
        if path in g.nodes[node_id]['name'] and len(g.nodes[node_id]['name'].split(path)[-1])==0:
            if node_type is not None:
                if g.nodes[node_id]['type'] == node_type:
                    node_ids.append(node_id)
            else:
                node_ids.append(node_id)
    return node_ids

def get_child_node_by_line(g: nx.DiGraph, parent_node_id:str, line=int):
    parent_node_type = g.nodes[parent_node_id]['type']
    assert parent_node_type in ['class', 'function'], f"Invalid node type {parent_node_type}"
    
    for child_node_id in list(g[parent_node_id]):
        child_node_attr = g.nodes[child_node_id]
        if child_node_attr.get('start_line',float('inf')) <= line <= child_node_attr.get('end_line',float('-inf')):
            return child_node_id
    return None
    

def detach_node(g: nx.DiGraph, node_id:str):
    """
    Deletes a node and all its descendants from a directed graph.

    Parameters:
    g (nx.DiGraph): The directed graph.
    node_name (str): The name of the node to delete.

    Returns:
    None: The graph is modified in place.
    """
    if node_id not in g:
        print(f"Node {node_id} not found in the graph.")
        return
    
    # Get all descendants of the node
    descendants = nx.descendants(g, node_id)
    
    # Add the node itself to the list of nodes to remove
    nodes_to_remove = descendants.union({node_id})
    
    # Remove the nodes from the graph
    g.remove_nodes_from(nodes_to_remove)

def get_parent_file(g: nx.DiGraph, node_id: str):
    """
    Finds the unique parent node n-hops away with an attribute 'type' equal to 'file'.

    Parameters:
    g (nx.DiGraph): The directed graph.
    node_name (str): The name of the node to start the search.

    Returns:
    str or None: The name of the parent node with 'type' == 'file', or None if no such node exists.
    """
    if node_id not in g:
        print(f"Node {node_id} not found in the graph.")
        return None

    # Use a queue to perform BFS-like traversal upwards
    current_node_ids = list(g.predecessors(node_id))

    visited = set()

    while current_node_ids:
        next_node_ids = []
        for parent_id in current_node_ids:
            if parent_id in visited:
                continue
            visited.add(parent_id)

            # Check if the current parent has the required attribute
            if g.nodes[parent_id].get('type') == 'file':
                return parent_id

            # Add the parents of the current node to the next level
            next_node_ids.extend(g.predecessors(parent_id))
        
        current_node_ids = next_node_ids

    # No parent with 'type' == 'file' found
    return None


def modify_node(g: nx.DiGraph, node_id: str, source: str):
    """
    Modify the content of a node in the graph by updating its source code.

    Parameters:
    g (nx.DiGraph): The directed graph.
    node_name (str): The name of the node to modify.
    source (str): The new source code for the node.

    Returns:
    nx.DiGraph: The updated graph.
    """
    assert g.nodes[node_id]['type'] in ['class', 'function'], f"Invalid node type {g.nodes[node_id]['type']}"

    
    # Get the node's start and end lines
    node_attr = g.nodes[node_id]
    start_line, end_line, indent = node_attr['start_line'], node_attr['end_line'], node_attr['indent']

    # Find the parent file containing the source
    parent_node_id = get_parent_file(g, node_id)
    parent_node_attr = g.nodes[parent_node_id]
    if parent_node_id is None:
        raise ValueError(f"Parent file not found for node {node_id}")

    # Get the parent file's source
    file_source = parent_node_attr['source']
    file_lines = file_source.splitlines(keepends=True)  # Preserve line endings

    # Apply indentation to the new source code
    indent_str = ' ' * indent  # Adjust this if tabs are used instead
    indented_source = ''.join(f"{indent_str}{line}\n" for line in source.splitlines())

    # Modify the source by replacing the node's lines
    parent_new_source = (
        ''.join(file_lines[:start_line - 1]) +  # Lines before the node
        indented_source +  # Indented new source code
        ''.join(file_lines[end_line:])  # Lines after the node
    )

    # Detach the current node
    detach_node(g, parent_node_id)

    # Update the parent node's source and reconstruct the graph
    g = add_nodes_by_source(g, parent_node_attr['name'], parent_new_source)
    g = remove_duplicates(g)

    return g
    





In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()